In [59]:
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
class Graph:
    def __init__(self, num_cities, dist_matrix=None, low=3, high=50):
        self.num_cities = num_cities
        if dist_matrix is not None:
            self.dist_matrix = dist_matrix
        else:
            self.dist_matrix = self.generate_distances(num_cities, low, high)

    '''
    for ex
    [[ 0  6  8  5]
     [ 6  0 10  4]
     [ 8 10  0  9]
     [ 5  4  9  0]]

    '''
    def generate_distances(num_cities, low=3, high=50):
        dist = np.zeros((num_cities, num_cities), dtype=int)
        for i in range(num_cities):
            for j in range(i + 1, num_cities):
                distance = random.randint(low, high)
                dist[i, j] = dist[j, i] = distance
        return dist

    '''
        path = [0, 2, 3, 1]

        cost = 0
        cost += dist[0][2] = 8
        cost += dist[2][3] = 9
        cost += dist[3][1] = 4
        cost += dist[1][0] = 6
        '''
    def path_cost(path, dist_matrix):
        cost = 0
        for i in range(len(path) - 1):
            cost += dist_matrix[path[i], path[i + 1]]
        cost += dist_matrix[path[-1], path[0]]  # Return to start
        return cost

In [62]:
class Ant:
    def __init__(self, graph:Graph, pheromone, alpha=1, beta=5):
        ''''
            3. Create a graph representation of the potential solutions, where each node
              represents a location and each edge represents a path between two locations.
         '''
        self.graph = graph
        self.dist_matrix = graph.dist_matrix
        self.num_cities = graph.num_cities

        self.pheromone = pheromone
        self.alpha = alpha
        self.beta = beta
        self.path = []
        self.visited = set()


    def select_next_city(self):
        '''
        b. For each ant, choose a path to follow from the starting point to the endpoint
        using a probabilistic decision rule that is based on the pheromone levels on the edges.
         prob = (τ^α * η^β) / ∑(τ^α * η^β)
        '''
        current = self.path[-1] # start with the last city in the path now

        unvisited = [city for city in range(self.num_cities) if city not in self.visited]
        if not unvisited:
            return None

        probabilities = []

        #∑(τ^α⋅η^β).
        denominator = 0
        for city in unvisited:
            tau = self.pheromone[current][city] ** self.alpha
            eta = (1 / self.dist_matrix[current][city]) ** self.beta
            denominator += tau * eta

        for city in unvisited:
            tau = self.pheromone[current][city] ** self.alpha
            eta = (1 / self.dist_matrix[current][city]) ** self.beta
            prob = (tau * eta) / denominator if denominator > 0 else 0
            probabilities.append(prob)

        # Roulette Wheel Selection
        r = random.random()
        cumulative = 0
        for city, prob in zip(unvisited, probabilities):
            cumulative += prob
            if r <= cumulative:
                return city
        return unvisited[-1] # default to the last city if no selection is made


    def build_full_path(self):
        start = random.randint(0, self.num_cities - 1)
        self.path = [start]
        self.visited = {start}
        while len(self.path) < self.num_cities:
            next_city = self.select_next_city()
            if next_city is None:
                break
            self.path.append(next_city)
            self.visited.add(next_city)

    def complete_path(self):
        while len(self.visited) < self.num_cities:
            next_city = self.select_next_city()
            if next_city is None:
                break
            self.path.append(next_city)
            self.visited.add(next_city)


In [63]:

class Colony:
    ''''
        1. Initialize the parameters of the algorithm, including the number of Ants M,
    the pheromone evaporation rate ρ, and the initial pheromone level on
    the edges τxy .
    '''
    def __init__(self,  graph: Graph, num_ants, alpha=1, beta=5, rho=0.1, Q=100):
        self.graph = graph
        self.dist_matrix = graph.dist_matrix
        self.num_cities = graph.num_cities

        self.num_ants = num_ants
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.Q = Q
        self.pheromone = np.ones((self.num_cities, self.num_cities))  # τ_xy = 1 at first
        self.ants = []
        self.best_path = None
        self.best_cost = float('inf')
        self.pheromone_snapshots = []
        self.best_paths_snapshots = []

    def create_ants(self):
        '''
            a. Release a group of artificial ants onto the graph, where each ant is placed at the starting point.
        '''
        self.ants = [Ant(self.graph, self.pheromone, self.alpha, self.beta)
                 for _ in range(self.num_ants)]


    def update_pheromones(self):
        '''
        c. Update the pheromone levels on the edges based on the quality of
        the solutions found by the ants. The pheromone level on each edge is increased by
        delta[x, y] += self.Q / L

        d. Evaporate the pheromone on each edge by a fixed amount, to prevent the pheromone from accumulating indefinitely and allow the ants to adapt to changing conditions.
        '''
        delta = np.zeros_like(self.pheromone)
        for ant in self.ants:
            L = Graph.path_cost(ant.path, self.dist_matrix)
            for i in range(len(ant.path) - 1):
                x, y = ant.path[i], ant.path[i + 1]
                delta[x, y] += self.Q / L
                delta[y, x] += self.Q / L

            x, y = ant.path[-1], ant.path[0] # Return to start
            delta[x, y] += self.Q / L
            delta[y, x] += self.Q / L

        self.pheromone = self.pheromone * (1 - self.rho) + delta

    def run(self, iterations=50):
        '''
        e. Determine whether a stopping criterion has been met, such as a maximum number of iterations or a satisfactory level of solution quality. Algorithm Steps
        '''
        best_costs_per_iteration = []
        average_costs_per_iteration = []

        for it in range(iterations):
            self.create_ants()

            for ant in self.ants:
                ant.build_full_path()

            self.update_pheromones()

            avg_cost = np.mean([Graph.path_cost(ant.path, self.dist_matrix) for ant in self.ants])
            average_costs_per_iteration.append(avg_cost)

            for ant in self.ants:
                cost = Graph.path_cost(ant.path, self.dist_matrix)
                if cost < self.best_cost:
                    '''
                    6. Select the best solution found by the ants, which corresponds to the path with the strongest pheromone trail on the graph.
                    '''
                    self.best_cost = cost
                    self.best_path = ant.path.copy()

            best_costs_per_iteration.append(self.best_cost)

            if (it + 1) % 10 == 0:
                print(f"Iteration {it + 1}/{iterations}, Best cost: {self.best_cost}")
                self.pheromone_snapshots.append(self.pheromone.copy())
                self.best_paths_snapshots.append((self.best_path.copy(), self.best_cost))

        return self.best_path, self.best_cost, best_costs_per_iteration, average_costs_per_iteration


# Usage example and testing


In [ ]:
dist_10 = Graph.generate_distances(10)
dist_20 = Graph.generate_distances(20)

graph_10 = Graph(10, dist_10)
graph_20 = Graph(20, dist_20)

print("Distance matrix for 10 cities:")
print(dist_10)



In [ ]:
print("Distance matrix for 10 cities:")
print(dist_10)


In [ ]:
print("\nDistance matrix for 20 cities:")
print(dist_20)

In [ ]:
def plot_pheromone_snapshots(colony, num_cities):
    print(f"Visualizing pheromone snapshots for {num_cities} cities...")
    for i, pheromone_matrix in enumerate(colony.pheromone_snapshots):
        plt.figure(figsize=(8, 6))
        sns.heatmap(pheromone_matrix, annot=False, cmap='viridis')
        plt.title(f'Pheromone Matrix after iteration {(i + 1) * 10} ({num_cities} cities)')
        plt.xlabel('City j')
        plt.ylabel('City i')
        plt.show()

## Results for 10 cities


In [ ]:
ants_list = [1, 5, 10, 20]

print("Results for 10 cities:")

# Dictionary to store costs for plotting
best_costs_dict = {}
avg_costs_dict = {}

for n_ants in ants_list:
    colony = Colony(graph_10, num_ants=n_ants)
    best_path, best_cost, best_costs_list, avg_costs_list = colony.run()
    print(f"Ants: {n_ants} -> Best path cost: {best_cost}")
    best_costs_dict[n_ants] = best_costs_list
    avg_costs_dict[n_ants] = avg_costs_list

In [ ]:
# Plot 1: Best cost per iteration
plt.figure(figsize=(10, 6))
for n_ants in ants_list:
    plt.plot(best_costs_dict[n_ants], label=f'Best (Ants = {n_ants})')
plt.title('Best Path Cost Progression (10 cities)')
plt.xlabel('Iteration')
plt.ylabel('Best Cost')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plot 2: Average cost per iteration
plt.figure(figsize=(10, 6))
for n_ants in ants_list:
    plt.plot(avg_costs_dict[n_ants], linestyle='--', label=f'Avg (Ants = {n_ants})')
plt.title('Average Path Cost Progression (10 cities)')
plt.xlabel('Iteration')
plt.ylabel('Average Cost')
plt.legend()
plt.grid(True)
plt.show()


## Results for 20 cities

In [ ]:
colony_10 = Colony(graph_10, num_ants=10)
colony_10.run(iterations=50)
plot_pheromone_snapshots(colony, num_cities=10)

## Results for 20 cities

In [ ]:
print("\nResults for 20 cities:")

best_costs_dict = {}
avg_costs_dict = {}

for n_ants in ants_list:
    colony = Colony(graph_20, num_ants=n_ants)
    best_path, best_cost, best_costs_list, avg_costs_list = colony.run()
    print(f"Ants: {n_ants} -> Best path cost: {best_cost}")
    best_costs_dict[n_ants] = best_costs_list
    avg_costs_dict[n_ants] = avg_costs_list

In [ ]:
# Plot 1: Best cost per iteration
plt.figure(figsize=(10, 6))
for n_ants in ants_list:
    plt.plot(best_costs_dict[n_ants], label=f'Best (Ants = {n_ants})')
plt.title('Best Path Cost Progression (20 cities)')
plt.xlabel('Iteration')
plt.ylabel('Best Cost')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plot 2: Average cost per iteration
plt.figure(figsize=(10, 6))
for n_ants in ants_list:
    plt.plot(avg_costs_dict[n_ants], linestyle='--', label=f'Avg (Ants = {n_ants})')
plt.title('Average Path Cost Progression (20 cities)')
plt.xlabel('Iteration')
plt.ylabel('Average Cost')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
colony_20 = Colony(graph_20, num_ants=10)
colony_20.run(iterations=50)
plot_pheromone_snapshots(colony_20, 20)